# Walmart Sales: model validation, regularization and selection

In this notebook I selected the best regressor to predict weekly sales from the Wallmart Sales data[(Kaggle competition)](https://www.kaggle.com/competitions/walmart-sales-forecasting/overview).  
 
 In more detail, I:  
- Validated a first linear regressor I trained on the data:  
    - Evaluated **model performance** throught a **cross validation**  
    - **Optimized** my linear **model** via **regularization** and **grid search** for hypeparameter tuning
- Compared my linear model to another regressor: Random Forest Regressor


## Table of Contents  
1. Linear regressor: cross validation
2. Linear regressor: regularization
3. Random Forest Regressor 
4. Best model selection  
5. Conclusions